# Exercises

In [6]:
from __future__ import division, print_function, unicode_literals
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import datasets

In [7]:
%matplotlib inline

In [ ]:
# Problem # 8
iris=datasets.load_iris()
X=iris['data'][:,(2,3)]
y=iris['target']
setosa_or_versicolor = (y == 0) | (y == 1)
X = X[setosa_or_versicolor]
y = y[setosa_or_versicolor]

In [4]:
from sklearn.svm import SVC,LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
_c=5
_loss='hinge'
_randomstate=42
lin_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler',StandardScaler()),
])
X_scaled=lin_pipeline.fit_transform(X)


In [ ]:
# Linear Classification
lin_clf=LinearSVC(loss=_loss,C=_c,random_state=_randomstate)
lin_clf.fit(X_scaled,y)
print("LinearSVC:                   ", lin_clf.intercept_, lin_clf.coef_)

In [ ]:
# SVC
svc_clf=SVC(kernel='linear',C=_c)
svc_clf.fit(X_scaled,y)
print("SVC:                   ", svc_clf.intercept_, svc_clf.coef_)

In [ ]:
# SGDC Classifier
alpha = 1 / (_c * len(X))

from sklearn.linear_model import SGDClassifier
sgdc_clf=SGDClassifier(loss=_loss, learning_rate="constant", eta0=0.001, alpha=alpha,
                        max_iter=100000, tol=-np.infty, random_state=_randomstate)
sgdc_clf.fit(X_scaled,y)
print("SGDC:                   ", sgdc_clf.intercept_, sgdc_clf.coef_)

In [ ]:
w1 = -lin_clf.coef_[0, 0]/lin_clf.coef_[0, 1]
b1 = -lin_clf.intercept_[0]/lin_clf.coef_[0, 1]
w2 = -svc_clf.coef_[0, 0]/svc_clf.coef_[0, 1]
b2 = -svc_clf.intercept_[0]/svc_clf.coef_[0, 1]
w3 = -sgdc_clf.coef_[0, 0]/sgdc_clf.coef_[0, 1]
b3 = -sgdc_clf.intercept_[0]/sgdc_clf.coef_[0, 1]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

lin_clf.fit(X_scaled, y)
svc_clf.fit(X_scaled, y)
sgdc_clf.fit(X_scaled, y)

# Transform the decision boundary lines back to the original scale
line1 = scaler.inverse_transform([[-10, -10 * w1 + b1], [10, 10 * w1 + b1]])
line2 = scaler.inverse_transform([[-10, -10 * w2 + b2], [10, 10 * w2 + b2]])
line3 = scaler.inverse_transform([[-10, -10 * w3 + b3], [10, 10 * w3 + b3]])

# Plot all three decision boundaries
plt.figure(figsize=(11, 4))
plt.plot(line1[:, 0], line1[:, 1], "k:", label="LinearSVC")
plt.plot(line2[:, 0], line2[:, 1], "b--", linewidth=2, label="SVC")
plt.plot(line3[:, 0], line3[:, 1], "r-", label="SGDClassifier")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs") # label="Iris-Versicolor"
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo") # label="Iris-Setosa"
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.legend(loc="upper center", fontsize=14)
plt.axis([0, 5.5, 0, 2])

plt.show()

In [1]:
# Problem #9
try:
    from sklearn.datasets import fetch_openml
    mnist = fetch_openml('mnist_784', version=1, cache=True)
except ImportError:
    from sklearn.datasets import fetch_mldata
    mnist = fetch_mldata('MNIST original')

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(mnist['data'],mnist['target'], test_size=0.2,random_state=42)

In [ ]:
from sklearn.svm import LinearSVC
lin_clf=None
lin_clf=LinearSVC(random_state=_randomstate)
lin_clf.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

In [8]:
scaler=StandardScaler()
X_train_scaled=scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled=scaler.fit_transform(X_test.astype(np.float32))

In [ ]:
lin_clf.fit(X_train_scaled,y_train)
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

In [9]:
from sklearn.svm import SVC
svc_clf=None
svc_clf=SVC(decision_function_shape="ovr",gamma="auto")
svc_clf.fit(X_train_scaled,y_train)
y_pred = svc_clf.predict(X_train_scaled)


NameError: name 'accuracy_score' is not defined

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y_pred)

0.9848928571428571

In [14]:
from scipy.stats import reciprocal, uniform
from sklearn.model_selection import RandomizedSearchCV
param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {
     'C': reciprocal(1, 10),
     'gamma': reciprocal(0.001, 0.1),
    }
  ]
random_search = RandomizedSearchCV(svc_clf, param_grid,n_iter=10, verbose=2, cv=3)
random_search.fit(X_train_scaled[:1000],y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=1.6165966448081257, gamma=0.00781686769460576 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. C=1.6165966448081257, gamma=0.00781686769460576, total=   0.9s
[CV] C=1.6165966448081257, gamma=0.00781686769460576 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV] .. C=1.6165966448081257, gamma=0.00781686769460576, total=   0.9s
[CV] C=1.6165966448081257, gamma=0.00781686769460576 .................
[CV] .. C=1.6165966448081257, gamma=0.00781686769460576, total=   0.8s
[CV] C=5.883246208718288, gamma=0.005805932211595853 .................
[CV] .. C=5.883246208718288, gamma=0.005805932211595853, total=   0.8s
[CV] C=5.883246208718288, gamma=0.005805932211595853 .................
[CV] .. C=5.883246208718288, gamma=0.005805932211595853, total=   0.8s
[CV] C=5.883246208718288, gamma=0.005805932211595853 .................
[CV] .. C=5.883246208718288, gamma=0.005805932211595853, total=   0.8s
[CV] C=1.603242773990047, gamma=0.001267244389565059 .................
[CV] .. C=1.603242773990047, gamma=0.001267244389565059, total=   0.6s
[CV] C=1.603242773990047, gamma=0.001267244389565059 .................
[CV] .. C=1.603242773990047, gamma=0.001267244389565059, total=   0.6s
[CV] C=1.603242773990047, gamma=0.001267244389565059 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   23.9s finished


RandomizedSearchCV(cv=3, estimator=SVC(gamma='auto'),
                   param_distributions=[{'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f843582b450>,
                                         'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f843582b2d0>}],
                   verbose=2)

In [15]:
random_search.best_estimator_

SVC(C=1.603242773990047, gamma=0.001267244389565059)

In [16]:
random_search.best_score_

0.8859877841913771

In [17]:
random_search.best_estimator_.fit(X_train_scaled, y_train)


SVC(C=1.603242773990047, gamma=0.001267244389565059)

In [18]:
y_pred = random_search.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9896607142857143

In [20]:
# Problem #10
from sklearn.datasets import fetch_california_housing
housing=fetch_california_housing()
X=housing['data']
y=housing['target']

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
lin_pipeline=Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('scaler',StandardScaler()),
])
X_train_scaled=lin_pipeline.fit_transform(X_train)
X_test_scaled=lin_pipeline.transform(X_test)

In [23]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=42)
lin_svr.fit(X_train_scaled, y_train)

/opt/anaconda3/envs/AI-Notebooks/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(random_state=42)

In [24]:
from sklearn.metrics import mean_squared_error
y_pred=lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.9641780189948642

In [25]:
np.sqrt(mse)

0.9819256687727764

In [26]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   4.6s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.6s remaining:    0.0s


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   4.7s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................
[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=   4.5s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=   4.1s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=   4.1s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=   4.3s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=   3.7s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=   3.7s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  2.6min finished


RandomizedSearchCV(cv=3, estimator=SVR(),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8415eefd90>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f8415eefbd0>},
                   random_state=42, verbose=2)

In [28]:
y_pred=rnd_search_cv.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse
np.sqrt(mse)

0.5727524770785359